## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

Aug. 18 Aug. 24


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,August-19-2025,Condo/Co-op,1320 High Point A Pl S Unit A,Delray Beach,FL,33445.0,225000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1320-Hi...,Beaches MLS,RX-11112032,N,Y,26.466781,-80.11196


In [8]:
print('Input county name:')
county = input()

Input county name:


 Palm Beach County


In [10]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [11]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [12]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [13]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-08-22    27
2025-08-20    22
2025-08-19    13
2025-08-21    11
2025-08-23     1
Name: count, dtype: int64

In [14]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [18]:
df_top_ten = df2.head(10)
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
51,PAST SALE,2025-08-20,Condo/Co-op,800 S Ocean Blvd #205,Boca Raton,FL,33432.0,7675000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11083811,N,Y,26.339913,-80.070857
77,PAST SALE,2025-08-20,Condo/Co-op,330 S Ocean Blvd Unit E1,Palm Beach,FL,33480.0,3517550.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/330-S-Oce...,Beaches MLS,RX-11040640,N,Y,26.703901,-80.033669
64,PAST SALE,2025-08-22,Condo/Co-op,2700 Donald Ross Rd #207,Palm Beach Gardens,FL,33410.0,2860000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11060179,N,Y,26.880648,-80.071850
49,PAST SALE,2025-08-22,Condo/Co-op,2773 S Ocean Blvd Ph -C,Palm Beach,FL,33480.0,2775000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2773-S-Oc...,Beaches MLS,RX-11085426,N,Y,26.621203,-80.039538
3,PAST SALE,2025-08-23,Condo/Co-op,176 Lake Dr #1202,Palm Beach Shores,FL,33404.0,2500000.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Shores/17...,Beaches MLS,RX-11118058,N,Y,26.780313,-80.039566
36,PAST SALE,2025-08-21,Condo/Co-op,2700 N Ocean Dr Unit 304b,Riviera Beach,FL,33404.0,1825000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11090586,N,Y,26.785112,-80.033355
73,PAST SALE,2025-08-22,Condo/Co-op,3720 S Ocean Blvd #610,Highland Beach,FL,33487.0,1700000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11056430,N,Y,26.408119,-80.066261
76,PAST SALE,2025-08-20,Condo/Co-op,545 Bay Colony Dr N,Juno Beach,FL,33408.0,1433000.0,4.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/545-Bay-C...,Beaches MLS,RX-11042622,N,Y,26.861525,-80.065647
16,PAST SALE,2025-08-20,Condo/Co-op,242 Tresana Blvd #70,Jupiter,FL,33478.0,1150000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/242-Tresana-...,Beaches MLS,RX-11099049,N,Y,26.926899,-80.155641
60,PAST SALE,2025-08-22,Condo/Co-op,111 SE 1st Ave #411,Delray Beach,FL,33444.0,1025000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/111-SE-...,Beaches MLS,RX-11066084,N,Y,26.458950,-80.071868


## Collect Agent Information

In [15]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }
        response = requests.get(base_url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()

        # Safely drill down, falling back to {} if any level is None or missing
        info = data.get('data') or {}
        panel = info.get('mainHouseInfoPanelInfo') or {}
        house = panel.get('mainHouseInfo') or {}

        # Extract listing agents
        listing_agents = house.get('listingAgents') or []
        list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName') if len(listing_agents) > 0 else None
        list_broker_name_1 = listing_agents[0].get('brokerName')           if len(listing_agents) > 0 else None
        list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName') if len(listing_agents) > 1 else None
        list_broker_name_2 = listing_agents[1].get('brokerName')           if len(listing_agents) > 1 else None

        # Extract buying agents
        buying_agents = house.get('buyingAgents') or []
        buy_agent_name_1  = buying_agents[0].get('agentInfo', {}).get('agentName') if len(buying_agents) > 0 else None
        buy_broker_name_1 = buying_agents[0].get('brokerName')             if len(buying_agents) > 0 else None
        buy_agent_name_2  = buying_agents[1].get('agentInfo', {}).get('agentName') if len(buying_agents) > 1 else None
        buy_broker_name_2 = buying_agents[1].get('brokerName')             if len(buying_agents) > 1 else None

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return { key: None for key in [
            'url', 'list_agent_name_1','list_broker_name_1',
            'list_agent_name_2','list_broker_name_2',
            'buy_agent_name_1','buy_broker_name_1',
            'buy_agent_name_2','buy_broker_name_2'
        ] }

In [19]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [20]:
response_df = pd.DataFrame(response_list)

In [21]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [22]:
df_top_ten = merged_df

In [23]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-08-20,Condo/Co-op,800 S Ocean Blvd #205,Boca Raton,FL,33432.0,7675000.0,4.0,4.5,...,-80.070857,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Todd David Lipsich,Luxury Partners Realty,None,None
1,PAST SALE,2025-08-20,Condo/Co-op,330 S Ocean Blvd Unit E1,Palm Beach,FL,33480.0,3517550.0,2.0,2.5,...,-80.033669,https://www.redfin.com/FL/Palm-Beach/330-S-Oce...,Patricia Mahaney,"Sotheby's Intl. Realty, Inc.",None,None,Maria Fanjul-Ryan,"Tina Fanjul Associates, Inc.",None,None
2,PAST SALE,2025-08-22,Condo/Co-op,2700 Donald Ross Rd #207,Palm Beach Gardens,FL,33410.0,2860000.0,3.0,3.5,...,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Non Agent,Non-Member Selling Office,None,None
3,PAST SALE,2025-08-22,Condo/Co-op,2773 S Ocean Blvd Ph -C,Palm Beach,FL,33480.0,2775000.0,3.0,3.0,...,-80.039538,https://www.redfin.com/FL/Palm-Beach/2773-S-Oc...,Scott Gordon,Douglas Elliman,Mindy Gordon,Douglas Elliman,Scott Gordon,Douglas Elliman,None,None
4,PAST SALE,2025-08-23,Condo/Co-op,176 Lake Dr #1202,Palm Beach Shores,FL,33404.0,2500000.0,2.0,3.0,...,-80.039566,https://www.redfin.com/FL/Palm-Beach-Shores/17...,None,None,None,None,None,None,None,None
5,PAST SALE,2025-08-21,Condo/Co-op,2700 N Ocean Dr Unit 304b,Riviera Beach,FL,33404.0,1825000.0,2.0,2.5,...,-80.033355,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Dermot OBrien,"Sotheby's Intl. Realty, Inc.",None,None,Brian Jacobs,"Network Property Group, LLC",None,None
6,PAST SALE,2025-08-22,Condo/Co-op,3720 S Ocean Blvd #610,Highland Beach,FL,33487.0,1700000.0,3.0,3.5,...,-80.066261,https://www.redfin.com/FL/Highland-Beach/3720-...,Nancy Ghen,Coldwell Banker/BR,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR,Judith Randon,Judith Randon Realty
7,PAST SALE,2025-08-20,Condo/Co-op,545 Bay Colony Dr N,Juno Beach,FL,33408.0,1433000.0,4.0,2.0,...,-80.065647,https://www.redfin.com/FL/Juno-Beach/545-Bay-C...,Ashley Barbaree,Sutter & Nugent LLC,Mark Nickol,Sutter & Nugent LLC,Crystal Condit,Waterfront Properties & Club C,Steven Condit,Waterfront Properties & Club C
8,PAST SALE,2025-08-20,Condo/Co-op,242 Tresana Blvd #70,Jupiter,FL,33478.0,1150000.0,3.0,2.5,...,-80.155641,https://www.redfin.com/FL/Jupiter/242-Tresana-...,Jennifer Burke,Coldwell Banker Realty,None,None,Kathleen Smuts,"WildFlower Real Estate, LLC",None,None
9,PAST SALE,2025-08-22,Condo/Co-op,111 SE 1st Ave #411,Delray Beach,FL,33444.0,1025000.0,2.0,2.0,...,-80.071868,https://www.redfin.com/FL/Delray-Beach/111-SE-...,Pascal Liguori,"Premier Estate Properties, Inc",Antonio Liguori,"Premier Estate Properties, Inc",Katherine Degroot,Lehmann Realty Inc.,None,None


## Current Week's Values

In [24]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

74


In [25]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$578,767


In [26]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$316


In [27]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$42,828,790


In [28]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           74              578767          316                     42828790.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [29]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
64
-----------
Input Previous Week Condo Average Sales Price:
491305
-----------
Input Previous Week Condo Average PSF:
328
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
31400000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [30]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [31]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [32]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [33]:
pd.set_option('display.max_columns',None)

In [34]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-20,Condo/Co-op,800 S Ocean Blvd #205,Boca Raton,FL,33432.0,7675000.0,4.0,4.5,Presidential Place,4747.0,NaN,1994.0,NaN,1617.0,7022.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11083811,N,Y,26.339913,-80.070857,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Todd David Lipsich,Luxury Partners Realty,None,None,1,orange


In [35]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [36]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-20,Condo/Co-op,800 S Ocean Blvd #205,Boca Raton,FL,33432.0,7675000.0,4.0,4.5,Presidential Place,4747.0,NaN,1994.0,NaN,1617.0,7022.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11083811,N,Y,26.339913,-80.070857,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Todd David Lipsich,Luxury Partners Realty,None,None,1,orange
1,PAST SALE,2025-08-20,Condo/Co-op,330 S Ocean Blvd Unit E1,Palm Beach,FL,33480.0,3517550.0,2.0,2.5,Three Thirty Ocean Condo,1996.0,NaN,1977.0,NaN,1762.0,3216.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/330-S-Oce...,Beaches MLS,RX-11040640,N,Y,26.703901,-80.033669,https://www.redfin.com/FL/Palm-Beach/330-S-Oce...,Patricia Mahaney,"Sotheby's Intl. Realty, Inc.",None,None,Maria Fanjul-Ryan,"Tina Fanjul Associates, Inc.",None,None,2,blue
2,PAST SALE,2025-08-22,Condo/Co-op,2700 Donald Ross Rd #207,Palm Beach Gardens,FL,33410.0,2860000.0,3.0,3.5,Azure,3282.0,NaN,2016.0,NaN,871.0,3428.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11060179,N,Y,26.880648,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Non Agent,Non-Member Selling Office,None,None,3,blue
3,PAST SALE,2025-08-22,Condo/Co-op,2773 S Ocean Blvd Ph -C,Palm Beach,FL,33480.0,2775000.0,3.0,3.0,Carlyle House Condo,2163.0,NaN,1973.0,NaN,1283.0,4252.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2773-S-Oc...,Beaches MLS,RX-11085426,N,Y,26.621203,-80.039538,https://www.redfin.com/FL/Palm-Beach/2773-S-Oc...,Scott Gordon,Douglas Elliman,Mindy Gordon,Douglas Elliman,Scott Gordon,Douglas Elliman,None,None,4,blue
4,PAST SALE,2025-08-23,Condo/Co-op,176 Lake Dr #1202,Palm Beach Shores,FL,33404.0,2500000.0,2.0,3.0,Palm Beach Shores,2662.0,NaN,2008.0,NaN,939.0,3913.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Shores/17...,Beaches MLS,RX-11118058,N,Y,26.780313,-80.039566,https://www.redfin.com/FL/Palm-Beach-Shores/17...,None,None,None,None,None,None,None,None,5,blue
5,PAST SALE,2025-08-21,Condo/Co-op,2700 N Ocean Dr Unit 304b,Riviera Beach,FL,33404.0,1825000.0,2.0,2.5,2700 North Ocean Condo,1920.0,NaN,2008.0,NaN,951.0,2842.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11090586,N,Y,26.785112,-80.033355,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Dermot OBrien,"Sotheby's Intl. Realty, Inc.",None,None,Brian Jacobs,"Network Property Group, LLC",None,None,6,blue
6,PAST SALE,2025-08-22,Condo/Co-op,3720 S Ocean Blvd #610,Highland Beach,FL,33487.0,1700000.0,3.0,3.5,Toscana West,2736.0,NaN,2002.0,NaN,621.0,3560.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11056430,N,Y,26.408119,-80.066261,https://www.redfin.com/FL/Highland-Beach/3720-...,Nancy Ghen,Coldwell Banker/BR,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR,Judith Randon,Judith Randon Realty,7,blue
7,PAST SALE,2025-08-20,Condo/Co-op,545 Bay Colony Dr N,Juno Beach,FL,33408.0,1433000.0,4.0,2.0,Bay Colony,1908.0,NaN,1988.0,NaN,751.0,1235.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/545-Bay-C...,Beaches MLS,RX-11042622,N,Y,26.861525,-80.065647,https://www.redfin.com/FL/Juno-Beach/545-Bay-C...,Ashley Barbaree,Sutter & Nugent LLC,Mark Nickol,Sutter & Nugent LLC,Crystal Condit,Waterfront Properties & Club C,Steven Condit,Waterfront Properties & Club C,8,blue
8,PAST SALE,2025-08-20,Condo/Co-op,242 Tresana Blvd #70,Jupiter,FL,33478.0,1150000.0,3.0,2.5,Jupiter Count

In [37]:
%store -r map_box_api_key

In [38]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=9,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [39]:
m.save('index.html')

## Data snagger

In [40]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [41]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-20,Condo/Co-op,800 S Ocean Blvd #205,Boca Raton,FL,33432.0,7675000.0,4.0,4.5,Presidential Place,4747.0,NaN,1994.0,NaN,1617.0,7022.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11083811,N,Y,26.339913,-80.070857,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Todd David Lipsich,Luxury Partners Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [42]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_08252025


# CREATE TEMPLATE 

In [43]:
muni_set = set(df_top_ten['CITY'])

In [44]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [45]:
df_top_ten.reset_index(inplace=True,drop=True)

In [46]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [47]:
top_sale

'Presidential Place at 800 S Ocean Blvd #205 in Boca Raton'

In [48]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [49]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [50]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-20,Condo/Co-op,800 S Ocean Blvd #205,Boca Raton,FL,33432.0,7675000.0,4.0,4.5,Presidential Place,4747.0,NaN,1994.0,NaN,1617.0,7022.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11083811,N,Y,26.339913,-80.070857,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Todd David Lipsich,Luxury Partners Realty,None,None,1,orange,Presidential Place at 800 S Ocean Blvd #205 in...
1,PAST SALE,2025-08-20,Condo/Co-op,330 S Ocean Blvd Unit E1,Palm Beach,FL,33480.0,3517550.0,2.0,2.5,Three Thirty Ocean Condo,1996.0,NaN,1977.0,NaN,1762.0,3216.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/330-S-Oce...,Beaches MLS,RX-11040640,N,Y,26.703901,-80.033669,https://www.redfin.com/FL/Palm-Beach/330-S-Oce...,Patricia Mahaney,"Sotheby's Intl. Realty, Inc.",None,None,Maria Fanjul-Ryan,"Tina Fanjul Associates, Inc.",None,None,2,blue,Three Thirty Ocean Condo at 330 S Ocean Blvd U...
2,PAST SALE,2025-08-22,Condo/Co-op,2700 Donald Ross Rd #207,Palm Beach Gardens,FL,33410.0,2860000.0,3.0,3.5,Azure,3282.0,NaN,2016.0,NaN,871.0,3428.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11060179,N,Y,26.880648,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Non Agent,Non-Member Selling Office,None,None,3,blue,Azure at 2700 Donald Ross Rd #207 in Palm Beac...
3,PAST SALE,2025-08-22,Condo/Co-op,2773 S Ocean Blvd Ph -C,Palm Beach,FL,33480.0,2775000.0,3.0,3.0,Carlyle House Condo,2163.0,NaN,1973.0,NaN,1283.0,4252.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2773-S-Oc...,Beaches MLS,RX-11085426,N,Y,26.621203,-80.039538,https://www.redfin.com/FL/Palm-Beach/2773-S-Oc...,Scott Gordon,Douglas Elliman,Mindy Gordon,Douglas Elliman,Scott Gordon,Douglas Elliman,None,None,4,blue,Carlyle House Condo at 2773 S Ocean Blvd Ph -C...
4,PAST SALE,2025-08-23,Condo/Co-op,176 Lake Dr #1202,Palm Beach Shores,FL,33404.0,2500000.0,2.0,3.0,Palm Beach Shores,2662.0,NaN,2008.0,NaN,939.0,3913.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Shores/17...,Beaches MLS,RX-11118058,N,Y,26.780313,-80.039566,https://www.redfin.com/FL/Palm-Beach-Shores/17...,None,None,None,None,None,None,None,None,5,blue,Palm Beach Shores at 176 Lake Dr #1202 in Palm...
5,PAST SALE,2025-08-21,Condo/Co-op,2700 N Ocean Dr Unit 304b,Riviera Beach,FL,33404.0,1825000.0,2.0,2.5,2700 North Ocean Condo,1920.0,NaN,2008.0,NaN,951.0,2842.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11090586,N,Y,26.785112,-80.033355,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Dermot OBrien,"Sotheby's Intl. Realty, Inc.",None,None,Brian Jacobs,"Network Property Group, LLC",None,None,6,blue,2700 North Ocean Condo at 2700 N Ocean Dr Unit...
6,PAST SALE,2025-08-22,Condo/Co-op,3720 S Ocean Blvd #610,Highland Beach,FL,33487.0,1700000.0,3.0,3.5,Toscana West,2736.0,NaN,2002.0,NaN,621.0,3560.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11056430,N,Y,26.408119,-80.066261,https://www.redfin.com/FL/Highland-Beach/3720-...,Nancy Ghen,Coldwell Banker/BR,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR,Judith Randon,Judith Randon Realty,7,blue,Toscana West at 3720 S Ocean Blvd #610 in High...
7,PAST SALE,2025-08-20,Condo/Co-op,545 Bay Colony Dr N,Juno Beach,FL,33408.0,1433000.0,4.0,2.0,Bay Colony,1908.0,NaN,1988.0,NaN,751.0,1235.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/545-Bay-C...,Beaches MLS

In [51]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name.strip()}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Presidential Place closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $1,025,000 to $7,675,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Delray Beach, Juno Beach, Palm Beach Shores, Jupiter, Highland Beach, Palm Beach Gardens, Boca Raton, Palm Beach, Riviera Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 74 condo sales totaling $42,828,790 million from Aug. 18 to Aug. 24. The previous week, brokers closed 64 condo sales totaling $

In [52]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [53]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [54]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [55]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-20,Condo/Co-op,800 S Ocean Blvd #205,Boca Raton,FL,33432.0,7675000.0,4.0,4.5,Presidential Place,4747.0,NaN,1994.0,NaN,1617.0,7022.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11083811,N,Y,26.339913,-80.070857,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Todd David Lipsich,Luxury Partners Realty,None,None,1,orange,Presidential Place at 800 S Ocean Blvd #205 in...
1,PAST SALE,2025-08-20,Condo/Co-op,330 S Ocean Blvd Unit E1,Palm Beach,FL,33480.0,3517550.0,2.0,2.5,Three Thirty Ocean Condo,1996.0,NaN,1977.0,NaN,1762.0,3216.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/330-S-Oce...,Beaches MLS,RX-11040640,N,Y,26.703901,-80.033669,https://www.redfin.com/FL/Palm-Beach/330-S-Oce...,Patricia Mahaney,"Sotheby's Intl. Realty, Inc.",None,None,Maria Fanjul-Ryan,"Tina Fanjul Associates, Inc.",None,None,2,blue,Three Thirty Ocean Condo at 330 S Ocean Blvd U...
2,PAST SALE,2025-08-22,Condo/Co-op,2700 Donald Ross Rd #207,Palm Beach Gardens,FL,33410.0,2860000.0,3.0,3.5,Azure,3282.0,NaN,2016.0,NaN,871.0,3428.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11060179,N,Y,26.880648,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Non Agent,Non-Member Selling Office,None,None,3,blue,Azure at 2700 Donald Ross Rd #207 in Palm Beac...
3,PAST SALE,2025-08-22,Condo/Co-op,2773 S Ocean Blvd Ph -C,Palm Beach,FL,33480.0,2775000.0,3.0,3.0,Carlyle House Condo,2163.0,NaN,1973.0,NaN,1283.0,4252.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2773-S-Oc...,Beaches MLS,RX-11085426,N,Y,26.621203,-80.039538,https://www.redfin.com/FL/Palm-Beach/2773-S-Oc...,Scott Gordon,Douglas Elliman,Mindy Gordon,Douglas Elliman,Scott Gordon,Douglas Elliman,None,None,4,blue,Carlyle House Condo at 2773 S Ocean Blvd Ph -C...
4,PAST SALE,2025-08-23,Condo/Co-op,176 Lake Dr #1202,Palm Beach Shores,FL,33404.0,2500000.0,2.0,3.0,Palm Beach Shores,2662.0,NaN,2008.0,NaN,939.0,3913.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Shores/17...,Beaches MLS,RX-11118058,N,Y,26.780313,-80.039566,https://www.redfin.com/FL/Palm-Beach-Shores/17...,None,None,None,None,None,None,None,None,5,blue,Palm Beach Shores at 176 Lake Dr #1202 in Palm...
5,PAST SALE,2025-08-21,Condo/Co-op,2700 N Ocean Dr Unit 304b,Riviera Beach,FL,33404.0,1825000.0,2.0,2.5,2700 North Ocean Condo,1920.0,NaN,2008.0,NaN,951.0,2842.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11090586,N,Y,26.785112,-80.033355,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Dermot OBrien,"Sotheby's Intl. Realty, Inc.",None,None,Brian Jacobs,"Network Property Group, LLC",None,None,6,blue,2700 North Ocean Condo at 2700 N Ocean Dr Unit...
6,PAST SALE,2025-08-22,Condo/Co-op,3720 S Ocean Blvd #610,Highland Beach,FL,33487.0,1700000.0,3.0,3.5,Toscana West,2736.0,NaN,2002.0,NaN,621.0,3560.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11056430,N,Y,26.408119,-80.066261,https://www.redfin.com/FL/Highland-Beach/3720-...,Nancy Ghen,Coldwell Banker/BR,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR,Judith Randon,Judith Randon Realty,7,blue,Toscana West at 3720 S Ocean Blvd #610 in High...
7,PAST SALE,2025-08-20,Condo/Co-op,545 Bay Colony Dr N,Juno Beach,FL,33408.0,1433000.0,4.0,2.0,Bay Colony,1908.0,NaN,1988.0,NaN,751.0,1235.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/545-Bay-C...,Beaches MLS

In [56]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Presidential Place closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $1,025,000 to $7,675,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Delray Beach, Juno Beach, Palm Beach Shores, Jupiter, Highland Beach, Palm Beach Gardens, Boca Raton, Palm Beach, Riviera Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 74 condo sales totaling $42,828,790 million from Aug. 18 to Aug. 24. The previous week, brokers closed 64 condo sales totaling $

In [57]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Boca-Raton/800-S-Ocean-Blvd-33432/unit-205/home/42325659


In [58]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Palm-Beach/330-S-Ocean-Blvd-33480/unit-1e/home/178429665


In [59]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Delray-Beach/111-SE-1st-Ave-33444/unit-411/home/101721339


In [60]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Jupiter/242-Tresana-Blvd-33478/unit-70/home/171489198


In [61]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/330-S-Ocean-Blvd-33480/unit-1e/home/178429665


In [62]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Jupiter/242-Tresana-Blvd-33478/unit-70/home/171489198


In [63]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-20,Condo/Co-op,800 S Ocean Blvd #205,Boca Raton,FL,33432.0,7675000.0,4.0,4.5,Presidential Place,4747.0,NaN,1994.0,NaN,1617.0,7022.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11083811,N,Y,26.339913,-80.070857,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Todd David Lipsich,Luxury Partners Realty,None,None,1,orange,Presidential Place at 800 S Ocean Blvd #205 in...
1,PAST SALE,2025-08-20,Condo/Co-op,330 S Ocean Blvd Unit E1,Palm Beach,FL,33480.0,3517550.0,2.0,2.5,Three Thirty Ocean Condo,1996.0,NaN,1977.0,NaN,1762.0,3216.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/330-S-Oce...,Beaches MLS,RX-11040640,N,Y,26.703901,-80.033669,https://www.redfin.com/FL/Palm-Beach/330-S-Oce...,Patricia Mahaney,"Sotheby's Intl. Realty, Inc.",None,None,Maria Fanjul-Ryan,"Tina Fanjul Associates, Inc.",None,None,2,blue,Three Thirty Ocean Condo at 330 S Ocean Blvd U...
2,PAST SALE,2025-08-22,Condo/Co-op,2700 Donald Ross Rd #207,Palm Beach Gardens,FL,33410.0,2860000.0,3.0,3.5,Azure,3282.0,NaN,2016.0,NaN,871.0,3428.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11060179,N,Y,26.880648,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Bradley Ball,Frankel Ball Realty LLC,Shannon Ball,Frankel Ball Realty LLC,Non Agent,Non-Member Selling Office,None,None,3,blue,Azure at 2700 Donald Ross Rd #207 in Palm Beac...
3,PAST SALE,2025-08-22,Condo/Co-op,2773 S Ocean Blvd Ph -C,Palm Beach,FL,33480.0,2775000.0,3.0,3.0,Carlyle House Condo,2163.0,NaN,1973.0,NaN,1283.0,4252.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2773-S-Oc...,Beaches MLS,RX-11085426,N,Y,26.621203,-80.039538,https://www.redfin.com/FL/Palm-Beach/2773-S-Oc...,Scott Gordon,Douglas Elliman,Mindy Gordon,Douglas Elliman,Scott Gordon,Douglas Elliman,None,None,4,blue,Carlyle House Condo at 2773 S Ocean Blvd Ph -C...
4,PAST SALE,2025-08-23,Condo/Co-op,176 Lake Dr #1202,Palm Beach Shores,FL,33404.0,2500000.0,2.0,3.0,Palm Beach Shores,2662.0,NaN,2008.0,NaN,939.0,3913.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Shores/17...,Beaches MLS,RX-11118058,N,Y,26.780313,-80.039566,https://www.redfin.com/FL/Palm-Beach-Shores/17...,None,None,None,None,None,None,None,None,5,blue,Palm Beach Shores at 176 Lake Dr #1202 in Palm...
5,PAST SALE,2025-08-21,Condo/Co-op,2700 N Ocean Dr Unit 304b,Riviera Beach,FL,33404.0,1825000.0,2.0,2.5,2700 North Ocean Condo,1920.0,NaN,2008.0,NaN,951.0,2842.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11090586,N,Y,26.785112,-80.033355,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Dermot OBrien,"Sotheby's Intl. Realty, Inc.",None,None,Brian Jacobs,"Network Property Group, LLC",None,None,6,blue,2700 North Ocean Condo at 2700 N Ocean Dr Unit...
6,PAST SALE,2025-08-22,Condo/Co-op,3720 S Ocean Blvd #610,Highland Beach,FL,33487.0,1700000.0,3.0,3.5,Toscana West,2736.0,NaN,2002.0,NaN,621.0,3560.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11056430,N,Y,26.408119,-80.066261,https://www.redfin.com/FL/Highland-Beach/3720-...,Nancy Ghen,Coldwell Banker/BR,Judith Randon,Judith Randon Realty,Nancy Ghen,Coldwell Banker/BR,Judith Randon,Judith Randon Realty,7,blue,Toscana West at 3720 S Ocean Blvd #610 in High...
7,PAST SALE,2025-08-20,Condo/Co-op,545 Bay Colony Dr N,Juno Beach,FL,33408.0,1433000.0,4.0,2.0,Bay Colony,1908.0,NaN,1988.0,NaN,751.0,1235.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/545-Bay-C...,Beaches MLS

## Time on Market Calculator

In [73]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 12, 23) ## List (Earlier) date
date2 = datetime(2025, 8, 20) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

240


## Clean CSV for Datawrapper Chart

In [65]:
chart_df = df_top_ten

In [66]:
chart_df = chart_df.fillna(" ")

In [67]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [68]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [69]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [70]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [71]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [72]:
chart_df['PPSF'].astype(float)

0    1617.0
1    1762.0
2     871.0
3    1283.0
4     939.0
5     951.0
6     621.0
7     751.0
8     378.0
9    1014.0
Name: PPSF, dtype: float64

In [74]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [75]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [76]:
csv_date_string = today.strftime("%m_%d_%Y")

In [77]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [78]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-08-20,Presidential Place,800 S Ocean Blvd #205 in Boca Raton,"$7,675,000","4,747","$1,617",1994,Senada Adzem Douglas Elliman,Todd David Lipsich Luxury Partners Realty
1,2025-08-20,Three Thirty Ocean Condo,330 S Ocean Blvd Unit E1 in Palm Beach,"$3,517,550","1,996","$1,762",1977,"Patricia Mahaney Sotheby's Intl. Realty, Inc.","Maria Fanjul-Ryan Tina Fanjul Associates, Inc...."
2,2025-08-22,Azure,2700 Donald Ross Rd #207 in Palm Beach Gardens,"$2,860,000","3,282",$871,2016,Bradley Ball Frankel Ball Realty LLC Shannon B...,Non Agent Non-Member Selling Office
3,2025-08-22,Carlyle House Condo,2773 S Ocean Blvd Ph -C in Palm Beach,"$2,775,000","2,163","$1,283",1973,Scott Gordon Douglas Elliman Mindy Gordon Doug...,Scott Gordon Douglas Elliman
4,2025-08-23,Palm Beach Shores,176 Lake Dr #1202 in Palm Beach Shores,"$2,500,000","2,662",$939,2008,,
5,2025-08-21,2700 North Ocean Condo,2700 N Ocean Dr Unit 304b in Riviera Beach,"$1,825,000","1,920",$951,2008,"Dermot OBrien Sotheby's Intl. Realty, Inc.","Brian Jacobs Network Property Group, LLC"
6,2025-08-22,Toscana West,3720 S Ocean Blvd #610 in Highland Beach,"$1,700,000","2,736",$621,2002,Nancy Ghen Coldwell Banker/BR Judith Randon Ju...,Nancy Ghen Coldwell Banker/BR Judith Randon Ju...
7,2025-08-20,Bay Colony,545 Bay Colony Dr N in Juno Beach,"$1,433,000","1,908",$751,1988,Ashley Barbaree Sutter & Nugent LLC Mark Nicko...,Crystal Condit Waterfront Properties & Club C ...
8,2025-08-20,Jupiter Country Club,242 Tresana Blvd #70 in Jupiter,"$1,150,000","3,042",$378,2019,Jennifer Burke Coldwell Banker Realty,"Kathleen Smuts WildFlower Real Estate, LLC"
9,2025-08-22,111 First Delray Condo,111 SE 1st Ave #411 in Delray Beach,"$1,025,000","1,011","$1,014",2019,"Pascal Liguori Premier Estate Properties, Inc ...",Katherine Degroot Lehmann Realty Inc.
